# Importador de Resultados da Megasena

Para fins educativos, eu criei um pequeno notebook que importa os dados dos jogos da megasena a partir do site da caixa (https://loterias.caixa.gov.br/Paginas/Download-Resultados.aspx).<br>
Não preciso dizer que o código é apresentado como é sem nenhuma garantia ou reponsabilidade de uso do mesmo.


In [1]:
import pandas as pd
import urllib.request
import ssl
import json
import itertools
import numpy as np
from multiprocessing import Process,  current_process
from bs4 import BeautifulSoup
from threading import Thread
from threading import current_thread
from collections import ChainMap

A Função abaixo serve para remover os espaços e quebras de linhas das celulas do BeautifulSoup.
Com certeza deve haver um jeito mais elegante de fazer

In [2]:
#
# Remove espaços e quebras de linha da celular do BeautifulSoup
#
def stripText(a):
    if a:
        return ''.join(a.text.split())

In [3]:
#
# Faz a requisição HTTP para o site da caixa
# 
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode    = ssl.CERT_NONE
#
# URL Abaixo é a que rendeiza a tabela apresentada no link: https://loterias.caixa.gov.br/Paginas/Download-Resultados.aspx
#
url = 'https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados?modalidade=Mega-Sena'
req = urllib.request.Request(url=url)
f   = urllib.request.urlopen(req,context=ctx)
js  = f.read().decode('utf-8')
js  = json.loads(js)


In [4]:
#
# Aqui a gente pega o HTML que veio no JSON da caixa e intepreta ele como HTML :)
#
soup = BeautifulSoup(js['html'])


### Sobre a estrutura de dados
A estrutura de dados que é retornada pela tabela é um tanto quanto quanto confusa, com tabelas dentro de tabelas, então o código abaixo lida com essa complexidade para tentar extrair de forma simples as colunas que são renderizadas no browser.<br>
No final desta célula , teremos uma variável 'head', com a lista dos campos e uma variável data, com as linhas e os valores dos campos.

In [5]:
#
# Extrai os resultados da Megasena do Site da Caixa :)
#
table  = soup.find('table')
#
# head vai conter a lista de colunas com nomes :)
#
head   = table.find_all('thead',recursive=False)
head   = head[0].findChildren('tr',recursive=False)
head   = head[0].findChildren('th',recursive=False)
head   = list(map(stripText,head))
bodies = table.find_all('tbody',recursive=False)
#
# data, é uma lista de de listas com todas as colunas
#
data   = []
for body in bodies:
    a     = body.find('tr',recursive=False)
    cells = a.findChildren('td',recursive=False)
    row_data  = []
    for cell in cells:
        text = cell.text
        text = ''.join(text.split())
        row_data.append(text)
    data.append(row_data)
    

### Resultado
Abaixo você pode ver a lista de resultados dos jogos imprimida na ordem, note que eu não fiz nenhum tratamento com relação ao tipo de dado, mas eu acho que serve como uma ponto de partida para seus estudos.
Ah sim!, eu só imprimi as colunas dos resultados, mas poderia ser impresso tudo.

In [6]:
#
# Imprimi todos os resultados por sorteio
#
countDict = {}
for result in data:
    numbers = result[2:8]
    for dezena in numbers:
        if not dezena in countDict:
            countDict[dezena]  = 1
        else:
            countDict[dezena] += 1
          
    #
    # Imprimi apenas os resultados dos jogos.
    #
    # print(result[0],result[2:8])
    #
    # Imprimi todos os dados da tabela
    #
    # print(result[0],result)

sortedDict = sorted(countDict.items(), key=lambda x:x[1],reverse=True)
for x in sortedDict:
    print('Number:[' + x[0] + '] Repeated: ['+str(x[1])+' Times]')
    

Number:[010] Repeated: [302 Times]
Number:[053] Repeated: [301 Times]
Number:[005] Repeated: [288 Times]
Number:[037] Repeated: [283 Times]
Number:[033] Repeated: [281 Times]
Number:[023] Repeated: [279 Times]
Number:[034] Repeated: [279 Times]
Number:[004] Repeated: [277 Times]
Number:[030] Repeated: [277 Times]
Number:[041] Repeated: [276 Times]
Number:[042] Repeated: [276 Times]
Number:[038] Repeated: [276 Times]
Number:[044] Repeated: [276 Times]
Number:[032] Repeated: [274 Times]
Number:[035] Repeated: [274 Times]
Number:[028] Repeated: [274 Times]
Number:[017] Repeated: [273 Times]
Number:[043] Repeated: [271 Times]
Number:[029] Repeated: [269 Times]
Number:[027] Repeated: [269 Times]
Number:[056] Repeated: [269 Times]
Number:[054] Repeated: [269 Times]
Number:[049] Repeated: [268 Times]
Number:[036] Repeated: [268 Times]
Number:[016] Repeated: [268 Times]
Number:[051] Repeated: [268 Times]
Number:[011] Repeated: [267 Times]
Number:[013] Repeated: [264 Times]
Number:[002] Repeate

In [7]:
#
# Agora vamos criar algumas combinações
#
duplas     = list(itertools.combinations(countDict.keys(), 2))
trinas     = list(itertools.combinations(countDict.keys(), 3))
quadruplas = list(itertools.combinations(countDict.keys(), 4))
quintuplas = list(itertools.combinations(countDict.keys(), 5))
sextuplas  = list(itertools.combinations(countDict.keys(), 6))

print(len(duplas))
print(len(trinas))
print(len(quadruplas))
print(len(quintuplas))
print(len(sextuplas))


1770
34220
487635
5461512
50063860


In [26]:
#
# Vamos testar por Ocorrencia qual grupo apresentou mais resultados
#
combinationCalc = {}
def saveResult(m,name):
    print("Saving %d Records In:[%s]" %(len(m),name))
    with open("results/result-"+name+".json", "w") as outfile:
        json.dump(m, outfile)

def checkMaches(combinations):
    m      = {}
    index  = 0
    print("%s - Starting Checking :%d Records " %(current_process().name,len(combinations)))
    for d in combinations:
        index +=1
        if index % 1000 == 0:
            perctDone = index / len(combinations)
            perctDone = perctDone * 100
            print("%s - Current:[%d]: %s de %d(%s) Found: [%d] Matches" % (current_process().name,index,'Processed',len(combinations),format(perctDone, ".2f"),len(m)))
        for sorteio in data.copy():         
            result = np.in1d(d, sorteio[2:8]).all()
            # result = False
            if (result):
                key = '-'.join(d)
                #
                # List is matched
                #
                # print('Matche:' + str(d))
                # print(sorteio[2:8],sorteio[0])
                if key not in m:
                    m[key] = 1
                else:
                    m[key] += 1
    saveResult(m,current_process().name)
    print("%s - Done Checking :%d Records Found Total:[%d] matches" %(current_process().name,len(combinations),len(m)))


In [ ]:
#
# Vamos quebrar as combinações em chunks
#

workload = np.array_split(quadruplas, 6)
threads  = []
threadId = 0
for w in workload:
    threadId+=1
    t = Process(target=checkMaches,args=(w,),name='Thread-'+str(threadId))
    threads.append(t)


for t in threads:
    t.start()

for t in threads:
    t.join()

Thread-1 - Starting Checking :81273 Records 
Thread-2 - Starting Checking :81273 Records Thread-3 - Starting Checking :81273 Records Thread-4 - Starting Checking :81272 Records 

Thread-5 - Starting Checking :81272 Records 

Thread-6 - Starting Checking :81272 Records 
Thread-5 - Current:[1000]: Processed de 81272(1.23) Found: [68] Matches
Thread-4 - Current:[1000]: Processed de 81272(1.23) Found: [67] Matches
Thread-3 - Current:[1000]: Processed de 81273(1.23) Found: [64] Matches
Thread-1 - Current:[1000]: Processed de 81273(1.23) Found: [90] Matches


In [19]:

combinationCalc

{}

In [16]:
combinationCalc

{}